In [6]:
import cv2 as cv
import numpy as np
import argparse
import time

def getFaceBox(net, frame, conf_threshold=0.7):
    frameOpencvDnn = frame.copy()
    frameHeight = frameOpencvDnn.shape[0]
    frameWidth  = frameOpencvDnn.shape[1]

    blob = cv.dnn.blobFromImage(frameOpencvDnn, 1.0, (300, 300), [104, 117, 123], True, False)

    net.setInput(blob)
    detections = net.forward()
    bboxes = []

    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > conf_threshold:
            x1 = int(detections[0, 0, i, 3] * frameWidth)
            y1 = int(detections[0, 0, i, 4] * frameHeight)
            x2 = int(detections[0, 0, i, 5] * frameWidth)
            y2 = int(detections[0, 0, i, 6] * frameHeight)

            bboxes.append([x1, y1, x2, y2])
            cv.rectangle(frameOpencvDnn, (x1, y1), (x2, y2),
                         (0, 255, 0), int(round(frameHeight/150)), 8)

    return frameOpencvDnn, bboxes


parser = argparse.ArgumentParser(description='Use this script to run age and gender recognition using OpenCV.')
parser.add_argument('--input', default="", help='Path to input image or video file. Skip this argument to capture frames from a camera.')

args, _ = parser.parse_known_args()

faceProto = "../xmltxt/opencv_face_detector.pbtxt"
faceModel = "../xmltxt/opencv_face_detector_uint8.pb"

ageProto = "../xmltxt/age_deploy.prototxt"
ageModel = "../xmltxt/age_net.caffemodel"

genderProto = "../xmltxt/gender_deploy.prototxt"
genderModel = "../xmltxt/gender_net.caffemodel"

MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
ageList = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']
genderList = ['Male', 'Female']

ageNet = cv.dnn.readNet(ageModel, ageProto)
genderNet = cv.dnn.readNet(genderModel, genderProto)
faceNet = cv.dnn.readNet(faceModel, faceProto)

cap = cv.VideoCapture(args.input if args.input else 1, cv.CAP_DSHOW)
padding = 20

while True:
    t = time.time()
    hasFrame, frame = cap.read()
    if not hasFrame or frame is None:
        break

    frameFace, bboxes = getFaceBox(faceNet, frame)

    if not bboxes:
        cv.imshow("Age Gender Demo", frameFace)
        if cv.waitKey(1) & 0xFF in (ord('q'), 27):
            break
        continue

    for bbox in bboxes:
        face = frame[
            max(0, bbox[1]-padding):min(bbox[3]+padding, frame.shape[0]-1),
            max(0, bbox[0]-padding):min(bbox[2]+padding, frame.shape[1]-1)
        ]

        if face.size == 0:
            continue

        blob = cv.dnn.blobFromImage(face, 1.0, (227, 227), MODEL_MEAN_VALUES, swapRB=False)

        genderNet.setInput(blob)
        genderPreds = genderNet.forward()
        gender = genderList[genderPreds[0].argmax()]
        # print("Gender : {}, conf = {:.3f}".format(gender, genderPreds[0].max()))

        ageNet.setInput(blob)
        agePreds = ageNet.forward()
        age = ageList[agePreds[0].argmax()]
        # print("Age : {}, conf = {:.3f}".format(age, agePreds[0].max()))

        label = "{},{}".format(gender, age)
        cv.putText(frameFace, label, (bbox[0], max(0, bbox[1]-10)),
                   cv.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2, cv.LINE_AA)

    cv.imshow("Age Gender Demo", frameFace)
    # print("time : {:.3f}".format(time.time() - t))

    if cv.waitKey(1) & 0xFF in (ord('q'), 27):
        break

cap.release()
cv.destroyAllWindows()

### Example 

In [ ]:
import cv2
import numpy as np

model = cv2.FaceDetectorYN_create('face_detection_yunet_2022mar.onnx', '', (320, 320), 0.9, 0.3, 5000)
cam = cv2.VideoCapture(0)
model_age = cv2.dnn.readNetFromCaffe("age_deploy.prototxt", "age_net.caffemodel")
model_gender = cv2.dnn.readNetFromCaffe("gender_deploy.prototxt", "gender_net.caffemodel")
age_list = [('0-2'), ('4-6'), ('8-12'), ('15-20'), ('25-32'), ('30-43'), ('48-53'), ('60-100')]
gender_list = ['Male', 'Female']

while True:
    ret, frame = cam.read()
    h, w, _ = frame.shape
    model.setInputSize((w, h))
    _, results = model.detect(frame)  

    if results is not None:
        for res in results:  
            x1, y1, x2, y2 = map(int, res[:4])  
            x1 = max(0, x1)
            y1 = max(0, y1)
            x2 = min(w, x2)
            y2 = min(h, y2)
            
            crop = frame[y1:y2, x1:x2] 
            if crop.size == 0:  
                continue
                
            blob = cv2.dnn.blobFromImage(crop, 1.0, (227, 227), 
                                         (78.4263377603, 87.7689143744, 114.895847746), 
                                         swapRB=False)
            
            model_age.setInput(blob)
            pred_age = model_age.forward()
            age = age_list[pred_age[0].argmax()]
            
            model_gender.setInput(blob)
            pred_gender = model_gender.forward()
            gender = gender_list[pred_gender[0].argmax()]
            
            text = f"{gender}, {age}"
            
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 255), 2)
            cv2.putText(frame, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 
                        0.8, (0, 255, 255), 2, cv2.LINE_AA)

    cv2.imshow('Camera', frame)
    if cv2.waitKey(1) == ord('q'):
        break

cam.release()
cv2.destroyAllWindows()